In [ ]:
prompts = []
num_prompts = 0
import json
import glob
import os
import sys
for f in glob.glob('../dataset_prepare/abc_dataset/*.abc'):
    is_header = True
    prompt_string = ''
    completion_string = ''
    showed_title = False
    song_file = open(f, 'r')
    lines = song_file.readlines()
    for line in lines:
        line = line.replace("$", "")
        line = line.replace("dc=", "")
        line = line.strip()
    
        if line.startswith("V:"):
            is_header = False
    
        if is_header:
            if line.startswith("X:") or line.startswith("C:"):
                prompt_string += line+"\n"
    
            if line.startswith("T:") and not showed_title:
                prompt_string += line+"\n"
                showed_title = True
        else:
            if not line.startswith("w:") and not line.startswith("V:"):
    
            # remove end of line comments
            index = line.rfind('%')
            if index > 0:
                line = line[:index].strip()
    
            # remove inline comments
            parts = line.split('"')
            newline = ""
            for i, p in enumerate(parts):
                if i%2 == 0:
                    newline += p
                elif not p.startswith("^"):
                    newline += '"' + p + '"'
            line = ' '.join(newline.split())
    
            completion_string += line+"\n"
    
        if line.startswith("V:"):
            is_header = False
  
    prompt_string = prompt_string.replace(":",": ")
    prompt_string = prompt_string.replace('"', "`")
    prompt_string = prompt_string.replace("\n"," $ ")
  
    completion_string = completion_string.replace('"', "`")
    completion_string = completion_string.strip().replace("\n"," $ ")
  
    sample = {
        "prompt": prompt_string + '<song>',
        "completion": ' ' + completion_string + '$ <end>'
    }
    
    if prompt_string not in prompts:
        prompts.append(prompt_string)
        num_prompts += 1
  
    with open('openai_train.jsonl', 'a') as of:
        json.dump(sample, of)
        of.write('\n')

In [ ]:
!openai tools fine_tunes.prepare_data -f openai_train.jsonl

In [ ]:
# replace OPENAI_API_KEY to your own API key
!export OPENAI_API_KEY="OPENAI_API_KEY"; openai api fine_tunes.create -t openai_train.jsonl --model curie --n_epochs 5